In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import neurd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
from mesh_tools import trimesh_utils as tu
from datasci_tools import ipyvolume_utils as ipvu

from neurd import neuron_visualizations as nviz
from pathlib import Path

# Load the appropriate volume data inferface object

In [4]:
from vdi_chen import volume_data_interface as vdi
vdi.set_parameters_for_directory_modules()

In [5]:
data_directory = "./Data"

In [6]:
vdi.meshes_directory = data_directory

# Pick the segment Id

In [7]:
segment_id = 75086130432907868
#segment_id = 74875092987018534
#segment_id = 80433193454844380 # large neuron

# Data Prep: Loading mesh and creating dummy synapse table

In [8]:
mesh = vdi.fetch_segment_id_mesh(
    segment_id,
    #meshes_directory = data_directory,
    plot = True
)
mesh

/usr/local/lib/python3.8/dist-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

<trimesh.Trimesh(vertices.shape=(264565, 3), faces.shape=(529210, 3))>

In [9]:
import numpy as np
import pandas as pd
from datasci_tools import pandas_utils as pu
from pathlib import Path

def mesh_vertices_sampling(
    mesh,
    sampling_rate = 0.05,
    n_samples = None,
    verbose = False):
    """
    Purpsoe
    -------
    perform a random sampling of vertices of a mesh

    Application
    -----------
    Can use those vertices for example synapse coordinates

    Example
    ------
        
    syn_coords = mesh_vertices_sampling(mesh,**kwargs)
    syn_coords
    """
    
    verts = np.array(mesh.vertices)
    vert_num = len(verts)
    if n_samples is None:
        n_samples = int(sampling_rate*vert_num)
    rand_int = np.unique(np.random.randint(0,vert_num,size = (n_samples,)))
    coords = verts[rand_int,:]
    return coords

def synapse_df_dummy(
    mesh,
    segment_id,
    syn_coords = None,
    segment_id_secondary = 12345,
    prepost =None,
    synapse_size = 100,
    **kwargs
    ):
    """
    Purpose
    -------
    To creae a dataframe with the following columns:

    'segment_id',
    'segment_id_secondary',
    'synapse_id',
    'prepost',
    'synapse_x',
    'synapse_y',
    'synapse_z',
    'synapse_size'
    """
        

    if syn_coords is None:
        syn_coords = mesh_vertices_sampling(mesh,**kwargs)

    syn_ids = np.arange(len(syn_coords)).astype('int')

    df = pd.DataFrame()
    df[['synapse_x','synapse_y','synapse_z']] = syn_coords
    df['segment_id'] = segment_id
    df['segment_id_secondary'] = segment_id_secondary
    df['synapse_id'] = syn_ids
    if prepost is None:
        #prepost = np.array(['presyn','postsyn'])[np.random.randint(0,1,(len(syn_ids),))]
        prepost = np.random.choice(['presyn','postsyn'],size = len(syn_ids), replace = True)
    df['prepost'] = prepost
    df['synapse_size'] = synapse_size
    df = df[[
        'segment_id',
        'segment_id_secondary',
        'synapse_id',
        'prepost',
        'synapse_x',
        'synapse_y',
        'synapse_z',
        'synapse_size'
    ]]
    return df
            
syn_df = synapse_df_dummy(mesh,segment_id)    
syn_df

,segment_id,segment_id_secondary,synapse_id,prepost,synapse_x,synapse_y,synapse_z,synapse_size
0,75086130432907868,12345,0,postsyn,1320479.625,1198169.875,96525.062500,100
1,75086130432907868,12345,1,postsyn,1319369.875,1199400.000,96275.156250,100
2,75086130432907868,12345,2,presyn,1314868.750,1200367.375,91880.109375,100
3,75086130432907868,12345,3,presyn,1312006.125,1200424.375,88742.367188,100
4,75086130432907868,12345,4,postsyn,1311798.000,1200420.000,88685.046875,100
...,...,...,...,...,...,...,...,...
12852,75086130432907868,12345,12852,presyn,1406820.250,1244190.000,138925.203125,100
12853,75086130432907868,12345,12853,postsyn,1407209.875,1244190.000,138450.156250,100
12854,75086130432907868,12345,12854,postsyn,1407390.375,1244160.125,138724.828125,100
12855,75086130432907868,12345,12855,postsyn,1407030.250,1244190.000,138300.218750,100


In [10]:
synapse_filepath = pu.df_to_csv(syn_df,Path(data_directory) / Path(f"{segment_id}_synapses.csv"))
pu.csv_to_df(synapse_filepath)
vdi.set_synapse_filepath(
    synapse_filepath
)

In [11]:
# Plotting the dummy synapses to see if they mapped well
coords = syn_df[['synapse_x','synapse_y','synapse_z']]
nviz.plot_objects(mesh,scatters=coords)

/usr/local/lib/python3.8/dist-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

In [12]:
from datasci_tools import pipeline

products = pipeline.PipelineProducts()
products

# Step 1: Decimation

In [13]:
"""
NOTE: an already decimated mesh is used for the example
in order to speed up time, that's why the mesh_dec
object will be overwritten with original mesh object
"""

"\nNOTE: an already decimated mesh is used for the example\nin order to speed up time, that's why the mesh_dec\nobject will be overwritten with original mesh object\n"

In [14]:
decimation_parameters = dict(
    decimation_ratio = 0.25,
)

In [15]:
import time

st = time.time()
mesh_decimated = tu.decimate(
    mesh,
    **decimation_parameters
)

print(f"Total time = {time.time() - st}")

xvfb-run -n 2169 -s "-screen 0 800x600x24" meshlabserver $@  -i /neurd_packages/neuron_mesh_tools/CONNECTS/Chen_Lab/temp/neuron_51627.off -o /neurd_packages/neuron_mesh_tools/CONNECTS/Chen_Lab/temp/neuron_51627_decimated.off -s /neurd_packages/neuron_mesh_tools/CONNECTS/Chen_Lab/temp/decimation_meshlab_25997247.mls
Total time = 15.455748319625854


In [16]:
# saving any parameters used or output products for later use
products.set_stage_attrs(
    stage = "decimation",
    attr_dict = dict(
        decimation_parameters = decimation_parameters,
        segment_id = segment_id,
    ),
    
)

print(products)

{
  'decimation':
  {
    'decimation_parameters':
    {
      'decimation_ratio':0.25,
    },
    'segment_id':75086130432907868,
  },
}



In [17]:
nviz.plot_objects(mesh_decimated)

Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

# Step 2: Soma Identification

In [18]:
from neurd import soma_extraction_utils as sm

soma_extraction_parameters = dict(
    #max_n_mesh_facer = 100
)

In [19]:
from neurd import neuron_visualizations as nviz
verbose = True

soma_products = sm.soma_indentification(
    mesh_decimated,
    verbose=verbose,
    **soma_extraction_parameters
)

Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 2500.0 
large_mesh_threshold_inner = 2500.0 
soma_size_threshold = 562.5 
soma_size_threshold_max = 75000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
max_mesh_sized_filtered_away = 22500.0
xvfb-run -n 8220 -s "-screen 0 800x600x24" meshlabserver $@  -i /neurd_packages/neuron_mesh_tools/CONNECTS/Chen_Lab/temp/neuron_88187.off -o /neurd_packages/neuron_mesh_tools/CONNECTS/Chen_Lab/temp/neuron_88187_remove_interior.off -s /neurd_packages/neuron_mesh_tools/CONNECTS/Chen_Lab/temp/remove_interior_113482.mls
removed temporary input file: /neurd_packages/neuron_mesh_tools/CONNECTS/Chen_Lab/temp/neuron_88187.off
removed temporary output file: /neurd_packages/neuron_mesh_tools/CONNECTS/Chen_Lab/temp/neuron_88187_remove_interior.off
/neurd_packages/neuron_mesh_tools/CONNECTS/Chen_Lab/temp/remove_interior_113482.mls is being deleted....
There were 1 total interior meshes
Pieces satisfying glia requirements 

  0%|          | 0/10 [00:00<?, ?it/s]

      ------ Found 1 viable somas: [0.880647]
Using Poisson Surface Reconstruction for watertightness in soma_volume_ratio
xvfb-run -n 5719 -s "-screen 0 800x600x24" meshlabserver $@  -i /neurd_packages/neuron_mesh_tools/CONNECTS/Chen_Lab/Poisson_temp/neuron_694954.off -o /neurd_packages/neuron_mesh_tools/CONNECTS/Chen_Lab/Poisson_temp/neuron_694954_poisson.off -s /neurd_packages/neuron_mesh_tools/CONNECTS/Chen_Lab/Poisson_temp/poisson_819770.mls
removed temporary input file: /neurd_packages/neuron_mesh_tools/CONNECTS/Chen_Lab/Poisson_temp/neuron_694954.off
removed temporary output file: /neurd_packages/neuron_mesh_tools/CONNECTS/Chen_Lab/Poisson_temp/neuron_694954_poisson.off
mesh.is_watertight = True
/neurd_packages/neuron_mesh_tools/CONNECTS/Chen_Lab/Poisson_temp/poisson_819770.mls is being deleted....
Inside sphere validater: ratio_val = 2.4050880795597904



 Total time for run = 34.565722942352295
Before Filtering the number of somas found = 1

---Performing Soma Mesh Backtrackin

  0%|          | 0/9 [00:00<?, ?it/s]

Largest hole before segmentation = 62912.79710860358, after = 89613.42520974492,

ratio = 1.4244069462537046, difference = 26700.628101141338
filtered_soma_list_components = [<trimesh.Trimesh(vertices.shape=(8581, 3), faces.shape=(17030, 3))>]
Not need to do a second pass because already found a soma


In [20]:
from datasci_tools import ipyvolume_utils as ipvu 
ipvu.plot_objects(
    meshes = soma_products.soma_meshes,# + soma_products.nuclei_meshes,
    meshes_colors="random",
)

Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

In [21]:
sm.plot_soma_products(
    mesh_decimated,
    soma_products = soma_products,
    verbose = True
)

# of somas = 1
# of glia = 0
# of nuclei = 1


Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

In [22]:
soma_products.soma_extraction_parameters = soma_extraction_parameters

In [23]:
products.set_stage_attrs(
    stage = "soma_identification",
    attr_dict = soma_products,
)

print(products)

{
  'decimation':
  {
    'decimation_parameters':
    {
      'decimation_ratio':0.25,
    },
    'segment_id':75086130432907868,
  },
  'soma_identification':
  {
    'glia_meshes':[],
    'nuclei_meshes':[<trimesh.Trimesh(vertices.shape=(4646, 3), faces.shape=(9304, 3))>],
    'soma_extraction_parameters':
    {
    },
    'soma_meshes':[<trimesh.Trimesh(vertices.shape=(8581, 3), faces.shape=(17030, 3))>],
    'soma_run_time':34.56571936607361,
    'soma_sdfs':array([0.880647]),
  },
}



# Step 2b: Saving off pipeline products

In [24]:
from datasci_tools import system_utils as su
su.save_object(
    products,"products_up_to_soma_stage"
)

Saved object at /neurd_packages/neuron_mesh_tools/CONNECTS/Chen_Lab/products_up_to_soma_stage.pkl
File size is 7.746181 MB
Cleared mesh: <trimesh.Trimesh(vertices.shape=(4646, 3), faces.shape=(9304, 3))>:140449609655728
Cleared mesh: <trimesh.Trimesh(vertices.shape=(8581, 3), faces.shape=(17030, 3))>:140449343478944


# Step 3: Decomposition

In [25]:
from neurd import neuron

neuron_obj = neuron.Neuron(
    mesh = mesh_decimated,
    segment_id = segment_id, # don't need this explicitely if segment_id is already in products
    pipeline_products = products,
    suppress_preprocessing_print=False,
    suppress_output=False,
)

neuron_obj

--- 0) Having to preprocess the Neuron becuase no preprocessed data
Please wait this could take a while.....
Skipping the hole filling
limb_remove_mesh_interior_face_threshold = 0
use_meshafterparty = True
Using pre-computed somas: soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(8581, 3), faces.shape=(17030, 3))>]
Soma List = [<trimesh.Trimesh(vertices.shape=(8581, 3), faces.shape=(17030, 3))>]
soma_mesh_list_centers = [array([1344937.54329332, 1205760.83160471,  139424.80647943])]
Getting Glia and Nuclei Pieces Subtracted Away 0.031423330307006836
 Splitting mesh after soma cancellation 0.11534929275512695
# of split_meshes = 7
 Containing Mesh Indices 0.006861686706542969
containing_mesh_indices = {0: 0}
 non_soma_touching_meshes 1.3113021850585938e-05
There were 6 pieces found after size threshold
 Finding inside pieces and non_soma_touching meshes 0.2932088375091553
soma_containing_meshes = {0: [0]}

-----Before combining multiple mesh pieces-----
soma_containing_meshes = {0: [0

  0%|                                                                                                                                                                                                                          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/19637 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.80it/s]


meshparty_segment_size = 100
Time for 1st pass MP skeletonization: 0.3312556743621826
connecting at the root
branches_touching_root = [12]
combining close nodes
combine_threshold = 700
length of Graph = 3820
After combining close endpoints max(kept_branches_idx) = 12, len(kept_branches_idx) = 13
empty_indices % = 0.0
 conflict_indices % = 0.014359052556673782


...

  0%|          | 0/9 [00:00<?, ?it/s]

Filtering away by face and skeletal length

Number of segments: 18



  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

Filtering away by face and skeletal length
width_name BEFORE processing = no_spine_median_mesh_center
After processing
width_name = no_spine_median_mesh_center, distance_by_mesh_center= True, no_spines = True, summary_measure= median
Working on limb L0 branch 15
    No spines and using precomputed width: 1089.9828968661784
Working on limb L0 branch 14
    No spines and using precomputed width: 828.8907338923282
Working on limb L0 branch 16
    current_width= 415.1344041740468
Working on limb L0 branch 12
    No spines and using precomputed width: 771.0632291031145
Working on limb L0 branch 13
    current_width= 386.10064140833373
Working on limb L0 branch 8
    No spines and using precomputed width: 779.6649030243232
Working on limb L0 branch 11
    No spines and using precomputed width: 246.53713053007215
Working on limb L0 branch 6
    No spines and using precomputed width: 817.1551299684313
Working on limb L0 branch 7
    No spines and using precomputed width: 435.1581110467687
Work

In [26]:
from neurd import neuron_visualizations as nviz

nviz.visualize_neuron(
    neuron_obj,
    limb_branch_dict="all"
)


 Working on visualization type: mesh

 Working on visualization type: skeleton


Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

# Calculate statistics

In [27]:
decomp_products = neuron_obj.calculate_decomposition_products(
    store_in_obj = True,
)

decomp_products

In [28]:
print(decomp_products)

    bbox_volume:2.92156645359312
    bbox_x_max:1457440.0
    bbox_x_max_soma_relative:112502.45670667756
    bbox_x_min:1217910.0
    bbox_x_min_soma_relative:-127027.54329332244
    bbox_y_max:1273610.0
    bbox_y_max_soma_relative:67849.16839529201
    bbox_y_min:1172330.0
    bbox_y_min_soma_relative:-33430.831604707986
    bbox_z_max:187607.0
    bbox_z_max_soma_relative:48182.19352056895
    bbox_z_min:67177.7
    bbox_z_min_soma_relative:-72247.10647943105
    branch_length_mean:22.71750615219005
    branch_length_median:14.663879011652615
    centroid_volume:1055.9502501835468
    centroid_x:1344937
    centroid_x_nm:1344937.5432933224
    centroid_y:1205760
    centroid_y_nm:1205760.831604708
    centroid_z:139424
    centroid_z_nm:139424.80647943105
    max_limb_n_branches:19
    max_limb_skeletal_length:402182.1425925456
    max_soma_area:627.1727959189939
    median_branch_length:13221.453
    n_branches:54
    n_error_limbs:0
    n_faces:132302
    n_limbs:6
    n_long_bra

In [29]:
print(neuron_obj.pipeline_products)

{
  'decimation':
  {
    'decimation_parameters':
    {
      'decimation_ratio':0.25,
    },
    'segment_id':75086130432907868,
  },
  'soma_identification':
  {
    'glia_meshes':[],
    'nuclei_meshes':[<trimesh.Trimesh(vertices.shape=(4646, 3), faces.shape=(9304, 3))>],
    'soma_extraction_parameters':
    {
    },
    'soma_meshes':[<trimesh.Trimesh(vertices.shape=(8581, 3), faces.shape=(17030, 3))>],
    'soma_run_time':34.56571936607361,
    'soma_sdfs':array([0.880647]),
  },
  'decomposition':
  {
    'bbox_volume':2.92156645359312,
    'bbox_x_max':1457440.0,
    'bbox_x_max_soma_relative':112502.45670667756,
    'bbox_x_min':1217910.0,
    'bbox_x_min_soma_relative':-127027.54329332244,
    'bbox_y_max':1273610.0,
    'bbox_y_max_soma_relative':67849.16839529201,
    'bbox_y_min':1172330.0,
    'bbox_y_min_soma_relative':-33430.831604707986,
    'bbox_z_max':187607.0,
    'bbox_z_max_soma_relative':48182.19352056895,
    'bbox_z_min':67177.7,
    'bbox_z_min_soma_relative

In [30]:
# vdi.save_neuron_obj(
#     neuron_obj,
#     verbose = True
# )

In [31]:
vdi.s

AttributeError: 'DataInterfaceChen' object has no attribute 's'

# Step 4A: Multi Soma Split Suggestions

In [ ]:
from neurd import neuron_visualizations as nviz
nviz.plot_soma_limb_concept_network(
    neuron_obj
)

In [ ]:
multi_soma_split_parameters = dict()

In [ ]:
_ = neuron_obj.calculate_multi_soma_split_suggestions(
    plot = True,
    store_in_obj = True,
    **multi_soma_split_parameters
)


# Step 4B: Multi Soma Split Execution

In [ ]:
neuron_list = neuron_obj.multi_soma_split_execution(
    verbose = False,
)

In [ ]:
neuron_list

In [ ]:
n1 = neuron_list[0]
print(n1.pipeline_products)

# Step 5: Cell Typing

In [ ]:
from neurd import neuron_pipeline_utils as npu

In [ ]:
neuron_obj_axon = npu.cell_type_ax_dendr_stage(
    n1,
    mesh_decimated = mesh_decimated,
    plot_axon = False,
)

In [ ]:
nviz.plot_axon(
    neuron_obj_axon
)

In [ ]:
print(neuron_obj_axon.pipeline_products.cell_type_ax_dendr)

# If the true synapses and a rotation matrix was not provided then the axon identification can't be trusted (and overwrit them)

In [ ]:
# showing that axon classification is stored in the branch labels
neuron_obj_axon[0][0].labels,neuron_obj_axon[1][0].labels

In [ ]:
def make_every_branch_default_label(neuron_obj):
    for limb in neuron_obj:
        for b in limb:
            b.labels = ['dendrite']

make_every_branch_default_label(neuron_obj_axon)
neuron_obj_dendrite = neuron_obj_axon

In [ ]:
nviz.plot_axon(
    neuron_obj_dendrite
)

# Saving off the neuron

In [ ]:
processed_dir = "./Data_Processed/"

In [ ]:
# Saving off the neuron object (and decimated mesh which is needed to read back in the saved object

suffix = "_cell_type"
vdi.save_neuron_obj(
    neuron_obj_dendrite,
    directory=processed_dir,
    suffix=suffix
)

In [ ]:
from mesh_tools import trimesh_utils as tu
tu.write_h5_file(
    mesh_decimated,
    filepath=processed_dir,
    filename = f"{segment_id}_decimated",
    return_file_path=True,
)

# Can continue to the cell typing ipynb with this processed mesh